## Predict and classify oxygen-binding proteins using random forest machine learning

describe

***

This project can be found at:
https://github.com/mccurcio/random_forest_dk_project.git

See also: /home/mcc/Dropbox/git_projects/random_forest_dk_project

In [6]:
dir_path = "/home/mcc/Dropbox/git_projects/random_forest_dk_project/uniprot_1/"

heme_files =  ("uniprot-erythrocruorin.fasta",
               "uniprot-hemerythrin.fasta",
               "uniprot-hemocyanin.fasta",
               "uniprot-hemoglobin.fasta",
               "uniprot-erythrocruorin.fasta",
               "uniprot-leghemoglobin.fasta",
               "uniprot-myoglobin.fasta")

number_files = len(heme_files)

print(number_files)
print(file_path)

7
/home/mcc/Dropbox/git_projects/random_forest_dk_project/uniprot_1/


In [ ]:
f = open ("myfile.fa")
seqs = {}

for for i in number_files:
for line in f:
    # Discard the newline (\n) at the end, if any
    line = line.rstrip() 
    # distinguish header from sequence
    if line[0] == '>': # or line.startswith('>')
        words=line.split()
        name = words[0][1:]
        seqs[name] = ‘’
    else:  # sequence, not header
        seqs[name] = seqs[name] + line
close(f)

In [10]:


def read_myaa(file_path, heme_files):
    for i in number_files:
        filepath = (dir_path + heme_files[i])
        f = open(filepath)
        count = 0
    with open("dna2.fasta") as f:
        for l in f:
            if l.startswith('>'):
                name = l[1:].rstrip('\n')
                count = count + 1
                sequences =[]
            else:
                sequences += l[:-1]
                output_file = open("out_" + str(count) + "_.txt", "w")
                output_file.write(str(len(sequences)))
    print count
    f.close

IndentationError: expected an indented block (<ipython-input-10-48bc5e1b8124>, line 5)

In [ ]:
# Load Fasta files for six classes of heme related proteins and clean

myfile = "uniprot-erythrocruorin.fasta"

try:
    f = open (myfile)
except IOError:
    print("File does not exist!")

seqs = {} # Use dictionary for proteins
for line in f:
    # let's discard the newline character (\n) at the end, if any
    line = line.rstrip() 
    # distinguish header from sequence
    if line[0] == '>': # or line.startswith('>')
        words = line.split()
        name = words[0][1:]
        seqs[name] = ''
    else:  # sequence, not header
        seqs[name] = seqs[name] + line
close(f)



In [ ]:
len(seqs)

# Print a dictionary
for x in seqs:
    print (x)
    for y in seqs[x]:
        print (y,':',seqs[x][y])

In [ ]:

# 7/11/16
# Script to remove duplicate FASTA reads based on Identifier:
# Given a FASTA file with some duplicate reads that share the same identifier,
# produce a valid FASTA with the identifier removed

def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]
    #Input is a list of tags, output is the unique items from the list, with the order of the list maintained
    #This function can be replaced by use of just the set() method if the order of the tags/FASTA does not need to be maintained


path_to_file = input("Enter path to FASTA as a string (use" "):")
path_to_file = path_to_file.replace('"','')
# e.g. /Users/User/Desktop/File.fas

File = open(path_to_file, 'r')
Text = File.read()

#Breaking FASTA into individual reads:
Lines = Text.split('>')
Lines = Lines[1:] #Removes the first empty item from the list
Tag_list = list()

for line in Lines:
    tags = line.split('\n')
    Tag_list.append(tags[0])

Sing_tags = f7(Tag_list)# Creates a list of all the unique tags

To_reconst = list()
for tag in Sing_tags:
    for line in Lines:
        if line.startswith(tag):
            To_reconst.append('>'+line)
            break
        else:
            pass

New_Fasta = ''.join(To_reconst)
File_2 = open('New_FASTA.fas','w')
File_2.write(str(New_Fasta))
File_2.close()

print ("Length of FASTA: ",len(Lines))
print ("Number of Unique Tags: ",len(Sing_tags))
print ("Length of New FASTA",len(To_reconst))


In [9]:
import matplotlib.pyplot as plt
import math
import argparse

# This lookup is used to correlate amino-acids to their
# corresponding Kyte-Doolittle hydropathy score.
hydropathy_lookup = {
    'A':  1.800,
    'R': -4.500,
    'N': -3.500,
    'D': -3.500,
    'C':  2.500,
    'Q': -3.500,
    'E': -3.500,
    'G': -0.400,
    'H': -3.200,
    'I':  4.500,
    'L':  3.800,
    'K': -3.900,
    'M':  1.900,
    'F':  2.800,
    'P': -1.600,
    'S': -0.800,
    'T': -0.700,
    'W': -0.900,
    'Y': -1.300,
    'V':  4.200,
}

# Make sure to at least set --file flag if you aren't working directly out of a BLAST search 
# (hence the 'seqdump.txt' default). Defaults are set for measuring 24TMS Ca & Na.
parser = argparse.ArgumentParser(description='Analyze proteins for a certain number of hydrophobic spanners')
parser.add_argument('--window', default=21, type=int, help='defines the width of the sliding window used')
parser.add_argument('--max', default=26, type=int, help='what is the maximum number of hydrophobic spanners that will be included in the passed group')
parser.add_argument('--min', default=22, type=int, help='what is the minimum number of hydrophobic spanners that will be included in the passed group')
parser.add_argument('--threshold', default=1.0, type=float, help='what hydropathy score is required to consider a spanner as hydrophobic')
parser.add_argument('--file', default='seqdump.txt', type=str, help='path to the data file')
parser.add_argument('--graph', action='store_true', help='should the spanners be graphed?')
parser.add_argument('--log', action='store_true', help='should we log things')
args = parser.parse_args().__dict__

# Variables in program set to argument values.
SLIDING_WINDOW_LENGTH = args['window']
MAX_HYDROPHOBIC_SPANNER_GROUPS = args['max']
MIN_HYDROPHOBIC_SPANNER_GROUPS = args['min']
HYDROPATHY_THRESHOLD = args['threshold']
GRAPH = args['graph']
LOG = args['log']
FILE = args['file']

# Map each amino-acid in the protein sequence to the average hydropathy
# of its SLIDING_WINDOW_LENGTH length of amino-acids.
def generate_spanners(protein_sequence):
    # Store the average hydropathy of each amino acids next SLIDING_WINDOW_LENGTH's
    # amount of amino acids.
    spanners = []

    # Loop over the protein_sequence to analyze each sliding window of amino
    # acids to determine average hydropathy.
    for idx, _ in enumerate(protein_sequence[0:-(SLIDING_WINDOW_LENGTH-1)]):

        # Retrieve spanner.
        spanner = protein_sequence[idx:(idx+SLIDING_WINDOW_LENGTH)]

        total_hydropathy = 0

        # Loop over the spanner to aggregate the hydropathy scores.
        for amino_acid in spanner:
            total_hydropathy += hydropathy_lookup[amino_acid]

        spanners.append(total_hydropathy/SLIDING_WINDOW_LENGTH)

    return spanners


if __name__ == "__main__":
    # Read the 'seqdump.txt' data file or whichever argument is flagged by '--file'.
    data_file = open(FILE, "r") 
    data_string = data_file.read() 

    # Uncomment to generate figure.
    plt.figure()

    protein_data = data_string.split("\n>")

    if LOG:
        print ('\n')
        print ('INPUT PROTEINS: \n\t' + str(len(protein_data)))
        print ('\n')

    passed_proteins = []

    # The data_string needs to be split on '\n>' to account for carrot
    # symbols being located in the protein descriptions.
    for datum_string in protein_data:

        # This handles the case of multiple lingering '>' in the description.
        if datum_string is "":
            continue

        # Separate the protein description and sequence using a new line delimiter.
        datum_lines = datum_string.split("\n", 1)
        protein_description = datum_lines[0]

        # The protein sequence needs all Xs removed as they can be intrepeted as
        # 'any' amino acid.
        protein_sequence = "".join(datum_lines[1].splitlines()).replace("X", "")

        # Given len(protein_sequence) number of SLIDING_WINDOW_LENGTHs calculate the average hydropathy value
        # for each sliding window.
        spanners = generate_spanners(protein_sequence)

        # Apply a value of 0 to all spanners that are less than HYDROPATHY_THRESHOLD. This will
        # allow us to later examine chains.
        hydrophobic_spanners = []
        for spanner in spanners:
            hydrophobic_spanners.append(spanner if spanner >= HYDROPATHY_THRESHOLD else 0)

        # This is the crux of the program. The algorithm uses idx+3 because when running initial data through,
        # the highest accuracy while still being inclusive of all 'actual' TMS was generated by this measure.
        hydrophobic_spanner_groups_count = 0
        for idx, spanner in enumerate(hydrophobic_spanners[:-(SLIDING_WINDOW_LENGTH+1)]):
            if spanner is not 0 and hydrophobic_spanners[idx+1] is not 0 and hydrophobic_spanners[idx+2] is not 0 and hydrophobic_spanners[idx+3] is 0:
                hydrophobic_spanner_groups_count += 1

        if hydrophobic_spanner_groups_count >= MIN_HYDROPHOBIC_SPANNER_GROUPS and hydrophobic_spanner_groups_count <= MAX_HYDROPHOBIC_SPANNER_GROUPS:
            passed_proteins.append({
                'description': protein_description,
                'sequence': protein_sequence,
                'hydrophobic_spanner_groups_count': hydrophobic_spanner_groups_count
            })

        if GRAPH:
            plt.plot([x for x in range(0, len(spanners)) if hydrophobic_spanners[x] >= HYDROPATHY_THRESHOLD], [x for x in hydrophobic_spanners if x >= HYDROPATHY_THRESHOLD])

    if GRAPH:
        plt.savefig("hydropathy-plot.png")

    if LOG:
        for protein in passed_proteins:
            print ('\n')
            print ('DESCRIPTION: \n\t' + protein['description'])
            print ('SEQUENCE LENGTH: \n\t' + str(len(protein['sequence'])))
            print ('SEQUENCE: \n\t' + protein['sequence'])

        print ('\n')
        print ('PROTEINS PASSING HYDROPHOBIC SPANNER GROUPS RANGE TEST: \n\t' + str(len(passed_proteins)))
        print ('\n')

    file = open("output.txt", "w") 
    file.write('\n'.join(['>'+protein['description']+'\n' + protein['sequence'] for protein in passed_proteins])) 
file.close() 

usage: ipykernel_launcher.py [-h] [--window WINDOW] [--max MAX] [--min MIN]
                             [--threshold THRESHOLD] [--file FILE] [--graph]
                             [--log]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-9cb305e5-f669-4108-b573-2933879dff2d.json


SystemExit: 2

/home/mcc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2855: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
